<a href="https://colab.research.google.com/github/u-khan902/starter-genai-langchain-py/blob/main/Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as numpy
import pandas as pd

In [3]:
df = pd.read_csv("/content/marketing_sample_for_walmart_com-ecommerce__20191201_20191231__30k_data.csv")


In [4]:
df1 = df.copy()
df = df.drop(columns = ['Crawl Timestamp','Product Url', 'Item Number', 'Gtin', 'Package Size','Postal Code'], axis =1)

In [5]:
df['Description'] = df['Description'].str.replace('We aim to show you accurate product information. Manufacturers, suppliers and others provide what you see here, and we have not verified it. See our disclaimer ',' ', regex = True)

In [6]:
!pip install ydata-profiling

ERROR: Operation cancelled by user


In [7]:
from ydata_profiling import ProfileReport
profile = ProfileReport(df, title = 'Profiling Report')
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/pool.py", line 856, in next
    item = self._items.popleft()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-193c3b2325bf>", line 3, in <cell line: 3>
    profile.to_notebook_iframe()
  File "/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py", line 520, in to_notebook_iframe
    display(get_notebook_iframe(self.config, self))
  File "/usr/local/lib/python3.10/dist-packages/ydata_profiling/report/presentation/flavours/widget/notebook.py", line 75, in get_notebook_iframe
    output = get_notebook_iframe_srcdoc(config, profile)
  File "/usr/local/lib/python3.10/dist-packages/ydata_profiling/report/presentation/flav

TypeError: object of type 'NoneType' has no len()

In [8]:
null_count = df.isnull().sum()
null_count

,0
Uniq Id,0
Product Name,0
Description,53
List Price,0
Sale Price,0
Brand,564
Category,17
Available,0


In [9]:
df = df.dropna()
df.isnull().sum()

,0
Uniq Id,0
Product Name,0
Description,0
List Price,0
Sale Price,0
Brand,0
Category,0
Available,0


In [10]:
!pip install langchain langchain_groq tiktoken pypdf unstructured[local-inference]

In [11]:
!pip install chromadb

In [12]:
import os
os.environ["GROQ_API_KEY"] = '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'
groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key

'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'

In [13]:
from langchain.prompts import PromptTemplate

# Define the template with placeholders for the product details
template = """
You are a product recommendation engine. Based on the following product details, generate a list of {top_k} similar products,
taking into account the product description, brand, category, and price range. Provide the product names and brief descriptions
of each recommendation.

Here are the details of the input product:

- Product Name: {product_name}
- Description: {description}
- Brand: {brand}
- Category: {category}
- List Price: {list_price}
- Sale Price: {sale_price}
- Availability: {availability}

Using this information, suggest {top_k} similar products, ensuring that they share common characteristics like category, brand,
or price range. For each recommendation, include:

1. Product Name
2. Brief Description
3. Price (if applicable)

### Example Output:
- Product Name: {{recommended_product_1}}
  - Description: {{recommended_product_description_1}}
  - Price: {{recommended_product_price_1}}

### Top {top_k} Recommendations:
"""

# Create a PromptTemplate object using the defined template
prompt = PromptTemplate(
    input_variables=["product_name", "description", "brand", "category", "list_price", "sale_price", "availability", "top_k"],
    template=template
)

# Example data (this would come from your dataset or user input)
product_data = {
    "product_name": "Samsung Galaxy S21",
    "description": "A high-performance smartphone with 8GB RAM, 128GB storage, 64MP camera, and 5G connectivity.",
    "brand": "Samsung",
    "category": "Smartphone",
    "list_price": "$999",
    "sale_price": "$799",
    "availability": "In Stock",
    "top_k": 3
}

# Generate the formatted prompt with actual data
formatted_prompt = prompt.format(**product_data)

# Display the formatted prompt
print(formatted_prompt)



You are a product recommendation engine. Based on the following product details, generate a list of 3 similar products, 
taking into account the product description, brand, category, and price range. Provide the product names and brief descriptions 
of each recommendation.

Here are the details of the input product:

- Product Name: Samsung Galaxy S21
- Description: A high-performance smartphone with 8GB RAM, 128GB storage, 64MP camera, and 5G connectivity.
- Brand: Samsung
- Category: Smartphone
- List Price: $999
- Sale Price: $799
- Availability: In Stock

Using this information, suggest 3 similar products, ensuring that they share common characteristics like category, brand, 
or price range. For each recommendation, include:

1. Product Name
2. Brief Description
3. Price (if applicable)

### Example Output:
- Product Name: {recommended_product_1}
  - Description: {recommended_product_description_1}
  - Price: {recommended_product_price_1}

### Top 3 Recommendations:



In [14]:
 from langchain.chains import LLMChain
 from langchain_groq import ChatGroq
 llm  = ChatGroq(groq_api_key = groq_api_key, model_name = 'Llama3-8b-8192')
 chain = LLMChain(llm = llm, prompt = prompt)

<ipython-input-14-ca3112bf9e95>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt)


In [15]:
!pip install -U langchain-community

In [16]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import chroma
from  langchain.chains import ConversationalRetrievalChain

,Uniq Id,Product Name,Description,List Price,Sale Price,Brand,Category,Available
0,019b67ef7f01103d8fb0a53e4c36daa7,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)",We aim to show you accurate product informati...,31.93,31.93,La Costeï¿½ï¿½a,"Food | Meal Solutions, Grains & Pasta | Canned...",True
1,3a4ff306dcc8a6e2bf720964d29b84c3,Equate Triamcinolone Acetonide Nasal Allergy S...,We aim to show you accurate product informati...,10.48,10.48,Equate,Health | Equate | Equate Allergy | Equate Sinu...,True
2,80090549d7d176327b186353c7b28ca4,AduroSmart ERIA Soft White Smart A19 Light Bul...,We aim to show you accurate product informati...,10.99,10.99,AduroSmart ERIA,Electronics | Smart Home | Smart Energy and Li...,True
3,151ee1c61a29bacfedb01cd500494b2f,"24"" Classic Adjustable Balloon Fender Set Chro...",We aim to show you accurate product informati...,38.59,38.59,lowrider,Sports & Outdoors | Bikes | Bike Accessories |...,True
4,7b2ef8d41f65df121f6b4b9828cf8dad,Elephant Shape Silicone Drinkware Portable Sil...,We aim to show you accurate product informati...,5.81,5.81,Anself,Baby | Feeding | Sippy Cups: Alternatives to P...,True


In [17]:

from langchain.docstore.document import Document

# Assuming your cleaned DataFrame is named 'df' and has the following columns:
# 'Uniq Id', 'Product Name', 'Description', 'List Price', 'Sale Price', 'Brand', 'Category', 'Available'

# Convert each row into a LangChain Document
documents = [
    Document(page_content=f"Uniq Id: {row['Uniq Id']}, "
                          f"Product Name: {row['Product Name']}, "
                          f"Description: {row['Description']}, "
                          f"List Price: {row['List Price']}, "
                          f"Sale Price: {row['Sale Price']}, "
                          f"Brand: {row['Brand']}, "
                          f"Category: {row['Category']}, "
                          f"Available: {row['Available']}")
    for _, row in df.iterrows()
]

# 'documents' now contains each row of the CSV file as a text-based document for further use


In [18]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
texts = text_splitter.split_documents(documents)

In [19]:
texts

[Document(metadata={}, page_content='Uniq Id: 019b67ef7f01103d8fb0a53e4c36daa7, Product Name: La Costena Chipotle Peppers, 7 OZ (Pack of 12), Description:   |La Costena Chipotle Peppers, 7 OZ (Pack of 12) Easy open. Ready to serve! Product of Mexico., List Price: 31.93, Sale Price: 31.93, Brand: La Costeï¿½ï¿½a, Category: Food | Meal Solutions, Grains & Pasta | Canned Goods | Canned Vegetables, Available: True'),
 Document(metadata={}, page_content='Uniq Id: 3a4ff306dcc8a6e2bf720964d29b84c3, Product Name: Equate Triamcinolone Acetonide Nasal Allergy Spray, 55 mcg per spray, 0.37 fl oz, Description:   |Compare to Nasacort Allergy 24HR active ingredient. Triamcinolone Acetonide Nasal Allergy Spray, 55 mcg per spray, allergy symptom reliever (glucocorticoid)*, is a non-drowsy, multi-symptom nasal spray for 24 hour relief of allergy symptoms such as nasal congestion, sneezing, runny and itchy nose. For adults and children ages 2 years and older. *Triamcinolone acetonide is a steroid medici

In [21]:
!pip install sentence-transformers

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from langchain.vectorstores import Chroma
kbstore = Chroma.from_documents(texts, embeddings)
print(kbstore)

In [ ]:
ret = kbstore.as_retriever(search_type = 'similarity', search_kwargs = {'k':3})

In [ ]:
model= ConversationalRetrievalChain.from_llm(llm = llm, retriever = ret)

In [ ]:
chat_history =[]
model({'question':'what is mention in the given document', 'chat_history':chat_history})